In [1]:
import pandas as pd
from pyspark import SparkContext,SQLContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

In [2]:
spark=SparkSession \
        .builder \
        .config("spark.eventLog.enabled", "false") \
        .config("spark.executor.memory", "4g")\
        .config("spark.driver.memory", "8g")\
        .config("spark.cores.max", "10")\
        .config("spark.task.maxFailures", "1000")\
        .config("spark.default.parallelism", "500")\
        .config("spark.sql.shuffle.partitions",100)\
        .appName('renhang_etl') \
        .master('yarn')\
        .getOrCreate()

In [3]:
# 定义udf 函数
def time_format(ts):
    try:
        res = ts[0:10].replace('.','')
        return res
    except:
        return 'None'
    else:
        return 'None'    
spark.udf.register('time_format',lambda x:time_format(x))

def format_year_month(time):
    try:
        res = time[0:7].replace('.','-')
        return res
    except:
        return 'None'
spark.udf.register('format_year_month',lambda x:format_year_month(x))

    
def get_time(ts):
    try:
        res = ts.replace('.','-')
        return res
    except:
        return 'None'
    else:
        return 'None'


spark.udf.register('get_time',lambda x:get_time(x))
# time_udfs = udf(get_time, StringType())


def timestampdiff(flag,t1,t2):
    try:
        date1 = t1[0:10].replace('.','-')
        date2 = t2[0:10].replace('.','-')
        year = (int(date1[0:4])-int(date2[0:4]))
        month = (int(date1[5:7])-int(date2[5:7]))
        day = int(date1[8:10])-int(date2[8:10])
        if(flag == 'year'):
            if(month<0 or (month==0 and day<0)):
                result = year -1
                return result
            return year
        if(flag == 'month'):
            res = year*12+month+int((day-30)/31)
            return res
    
    except:
        return 'None'
    
                                                                                      
spark.udf.register('timestampdiff',lambda x1,x2,x3:timestampdiff(x1,x2,x3))
# timestampdiff = udf(get_time, StringType())   


def get_city(address):
    try:
        province_loc = address.find("省")
        city_loc = address.find("市")
        if(province_loc < 0 or (province_loc>=0 and province_loc>=city_loc) ): #江西赣州市兴国县江西省赣州市兴国县潋江镇罗廖村26号
            city = address[0:city_loc+1]
        if(province_loc>=0 and province_loc<city_loc):
            city = address[province_loc+1:city_loc+1]
        return city
    except:
        return 'None'



spark.udf.register('get_city',lambda x:get_city(x))
# time_udfs = udf(get_time, StringType())
                                                                                    

In [4]:
#创建DW.dw_baseinfo_detail
table1 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRMESSAGEHEADER") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table1.createOrReplaceTempView("ICRMESSAGEHEADER")

table2 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRCREDITCUE") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table2.createOrReplaceTempView("ICRCREDITCUE")

table3 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLOANCARDINFO") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table3.createOrReplaceTempView("ICRLOANCARDINFO")

table4 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRCREDITCUE") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table4.createOrReplaceTempView("ICRCREDITCUE")

table5 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLATEST2YEAROVERDUECARD") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table5.createOrReplaceTempView("ICRLATEST2YEAROVERDUECARD")

table6 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLATEST5YEAROVERDUEDETAIL") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table6.createOrReplaceTempView("ICRLATEST5YEAROVERDUEDETAIL")

table7 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLATEST2YEAROVERDUE") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table7.createOrReplaceTempView("ICRLATEST2YEAROVERDUE")

table8 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRLOANINFO") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table8.createOrReplaceTempView("ICRLOANINFO")

table9 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRUNDESTORYLOANCARD") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table9.createOrReplaceTempView("ICRUNDESTORYLOANCARD")

table10 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRUNDESTORYSTANDARDLOANCARD") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table10.createOrReplaceTempView("ICRUNDESTORYSTANDARDLOANCARD")

table11 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRUNPAIDLOAN") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table11.createOrReplaceTempView("ICRUNPAIDLOAN")


table12 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICROVERDUESUMMARY") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table12.createOrReplaceTempView("ICROVERDUESUMMARY")


table13 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRFELLBACKSUMMARY") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table13.createOrReplaceTempView("ICRFELLBACKSUMMARY")


table14 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRGUARANTEE") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table14.createOrReplaceTempView("ICRGUARANTEE")


table15 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRGUARANTEESUMMARY") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table15.createOrReplaceTempView("ICRGUARANTEESUMMARY")


#######

table16 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRCIVILJUDGEMENT") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table16.createOrReplaceTempView("ICRCIVILJUDGEMENT")



table17 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRFORCEEXECUTION") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table17.createOrReplaceTempView("ICRFORCEEXECUTION")


table18 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRADMINPUNISHMENT") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table18.createOrReplaceTempView("ICRADMINPUNISHMENT")


table19 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRTAXARREAR") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table19.createOrReplaceTempView("ICRTAXARREAR")


table20 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRACCFUND") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table20.createOrReplaceTempView("ICRACCFUND")



table21 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRENDOWMENTINSURANCEDEPOSIT") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table21.createOrReplaceTempView("ICRENDOWMENTINSURANCEDEPOSIT")


table22 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRRECORDDETAIL") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table22.createOrReplaceTempView("ICRRECORDDETAIL")



table23 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRRECORDDETAIL") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table23.createOrReplaceTempView("ICRRECORDDETAIL")


table24 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.QUERY_HISTORY") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table24.createOrReplaceTempView("QUERY_HISTORY")

table25 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.87:3306") \
    .option("dbtable", "hyxf.ICRRECORDSUMMARY") \
    .option("user", "dt") \
    .option("password", "Usd&212%wePO2") \
    .load()
table25.createOrReplaceTempView("ICRRECORDSUMMARY")


###########

spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST2YEAROVERDUECARD ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where BIZTYPE='准贷记卡' and ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_standcard_2yearoverdue")


spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST2YEAROVERDUECARD ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where BIZTYPE='贷记卡' and ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_creditcard_2yearoverdue")


spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		ordue.LASTMONTHS,ordue.AMOUNT,loanfo.GUARANTEETYPE,STATE,FINANCEORG
from ICRLATEST5YEAROVERDUEDETAIL ordue
-- 只取贷记卡的信息
inner join ICRLOANCARDINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where month!='--'  and BIZTYPE='贷记卡'
""").createOrReplaceTempView("temp_creditcard_overdue")


spark.sql("""
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		MONTH,
		gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_creditcard_2yearoverdue
union 
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		format_year_month(concat(month,'.01')),
        gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_creditcard_overdue
""").createOrReplaceTempView("temp_creditcard_5yearoverdue")



spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		case 
			when ordue.LASTMONTHS in ('G','Z','D') then 8 
			else ordue.LASTMONTHS 
		end as LASTMONTHS ,
		loanfo.GUARANTEETYPE,
		STATE,
		FINANCEORG
from ICRLATEST2YEAROVERDUE ordue
-- 只取贷款的信息
inner join ICRLOANINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where ordue.LASTMONTHS!='C'
""").createOrReplaceTempView("temp_loan_2yearoverdue")



spark.sql("""
select 
		ordue.reportno,
		ordue.ACCOUNT,
		head.QUERYTIME,
		ordue.MONTH,
        timestampdiff('month', head.QUERYTIME, concat(ordue.month,'.01') ) as gap_months,
		ordue.LASTMONTHS,
		ordue.AMOUNT,
		loanfo.GUARANTEETYPE,
		STATE,FINANCEORG
from ICRLATEST5YEAROVERDUEDETAIL ordue
-- 只取贷款的信息
inner join ICRLOANINFO loanfo on ordue.reportno=loanfo.reportno and ordue.ACCOUNT=loanfo.ACCOUNT 
left join ICRMESSAGEHEADER head on ordue.reportno=head.reportno
where month!='--'
""").createOrReplaceTempView("temp_loan_overdue")




spark.sql("""
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
        MONTH,
		gap_months,
		LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_loan_2yearoverdue
union 
select 
		reportno,
		ACCOUNT,
		QUERYTIME,
		format_year_month(concat(month,'.01')),gap_months,LASTMONTHS,
		GUARANTEETYPE,
		STATE,
		FINANCEORG 
from temp_loan_overdue
""").createOrReplaceTempView("temp_loan_5yearoverdue")




In [5]:
spark.sql("""



select 
		head.reportno,
		if(cdt.FIRSTLOANCARDOPENMONTH='--' or cdt.FIRSTLOANCARDOPENMONTH is null,-9999999, timestampdiff('month', head.QUERYTIME, concat(FIRSTLOANCARDOPENMONTH,'.01') )   ) as first_lcard_months,
		if(cdt.FIRSTSTANDARDLOANCARDOPENMONTH='--' or cdt.FIRSTSTANDARDLOANCARDOPENMONTH is null,-9999999, timestampdiff('month', head.QUERYTIME, concat(FIRSTSTANDARDLOANCARDOPENMONTH,'.01') )  ) as first_scard_months,
		nvl(loan.lcard_max_months,-9999999) as lcard_max_months,
		nvl(loan.lcard_min_months,-9999999) as lcard_min_months,
		nvl(loan.lcard_avg_months,-9999999) as lcard_avg_months,
		nvl(lcdt.unlcard_max_months,-9999999) as unlcard_max_months,
		nvl(lcdt.unlcard_min_months,-9999999) as unlcard_min_months,
		nvl(lcdt.unlcard_avg_months,-9999999) as unlcard_avg_months,
		nvl(scard.scard_max_months,-9999999) as scard_max_months,
		nvl(scard.scard_min_months,-9999999) as scard_min_months,
		nvl(scard.scard_avg_months,-9999999) as scard_avg_months,
		nvl(unsc.unscard_max_months,-9999999) as unscard_max_months,
		nvl(unsc.unscard_min_months,-9999999) as unscard_min_months,
		nvl(unsc.unscard_avg_months,-9999999) as unscard_avg_months,
		nvl(unact.actlcard_max_months,-9999999) as actlcard_max_months,
		nvl(unact.actlcard_min_months,-9999999) as actlcard_min_months,
        now() as create_time,now() as update_time,now() as etl_date

from ICRMESSAGEHEADER head 
left join ICRCREDITCUE cdt on head.reportno=cdt.reportno
left join 
(
	select 
		a.reportno,
		max(cast(timestampdiff('month',b.QUERYTIME,OPENDATE) as int)) as lcard_max_months,
		min( cast(timestampdiff('month',b.QUERYTIME,OPENDATE) as int) ) as lcard_min_months,
		round(avg( cast(timestampdiff('month',b.QUERYTIME,OPENDATE) as int) ),5) as lcard_avg_months
	from ICRLOANCARDINFO a
	left join ICRMESSAGEHEADER b on a.reportno=b.reportno
	where biztype='贷记卡'
	group by a.reportno
)loan on head.reportno=loan.reportno
left join 
(
	select 
			a.reportno,
			max(cast(timestampdiff('month',b.QUERYTIME,OPENDATE) as int)) as unlcard_max_months,
			min(cast(timestampdiff('month',b.QUERYTIME,OPENDATE) as int) ) as unlcard_min_months,
			round(avg(cast(timestampdiff('month',b.QUERYTIME,OPENDATE) as int)),5) as unlcard_avg_months
	from ICRLOANCARDINFO a
	left join ICRMESSAGEHEADER b on a.reportno=b.reportno
	where state!='销户' and biztype='贷记卡'
	group by a.reportno
)lcdt on head.reportno=lcdt.reportno
left join 
(
	select 
		a.reportno,
		max( cast(timestampdiff('month',b.QUERYTIME,OPENDATE) as int) ) as scard_max_months,
		min( cast(timestampdiff('month',b.QUERYTIME,OPENDATE) as int) ) as scard_min_months,
		round(avg( cast(timestampdiff(b.QUERYTIME,OPENDATE) as int) ),5) as scard_avg_months
	from ICRLOANCARDINFO a
	left join ICRMESSAGEHEADER b on a.reportno=b.reportno
	where biztype='准贷记卡'
	group by a.reportno
)scard on head.reportno=scard.reportno
left join 
(
	select 
		a.reportno,
		max( cast(timestampdiff('month',b.QUERYTIME,OPENDATE) as int) ) as unscard_max_months,
		min( cast(timestampdiff('month',b.QUERYTIME,OPENDATE) as int) ) as unscard_min_months,
		round(avg( cast(timestampdiff(b.QUERYTIME,OPENDATE) as int) ),5) as unscard_avg_months 
	from ICRLOANCARDINFO a
	left join ICRMESSAGEHEADER b on a.reportno=b.reportno
	where state!='销户' and biztype='准贷记卡'
	group by a.reportno
)unsc on head.reportno=unsc.reportno
left join 
(
	select 
		a.reportno,
		max( cast(timestampdiff('month',b.QUERYTIME,OPENDATE) as int) ) as actlcard_max_months,
		min( cast(timestampdiff('month',b.QUERYTIME,OPENDATE) as int) ) as actlcard_min_months
	from ICRLOANCARDINFO a
	left join ICRMESSAGEHEADER b on a.reportno=b.reportno
	where state!='未激活' and biztype='贷记卡'
	group by a.reportno
)unact on head.reportno=unact.reportno


""").createOrReplaceTempView("dw_card_age")



In [6]:
table_name = "dw_card_age"

spark.sql("drop table if EXISTS renhang_user_profile.%s "%table_name)
spark.sql("create table renhang_user_profile.%s as select * from %s"%(table_name,table_name))

Py4JJavaError: An error occurred while calling o78.sql.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:211)
	at org.apache.spark.sql.hive.execution.InsertIntoHiveTable.run(InsertIntoHiveTable.scala:337)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:66)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:61)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.doExecute(commands.scala:84)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:117)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:117)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:138)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.Spark